hidden_size = 512

双端 GRU 模型，对所有的 timestep 采用attention 模型输出。

把 title 和 content 两部分分别放在两个网络中，在最后的 softmax 层之前拼起来。由于两部分的序列长度差的太多，所以跑起来会比较慢

参考代码： 

https://github.com/yongyehuang/deep-text-classifier/blob/master/HAN_model.py 其中attention最后加权的地方应该写错了

https://github.com/indiejoseph/doc-han-att/blob/master/model.py

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import pickle
import os
import sys
import shutil
import time


time0 = time.time()
print('Starting ...')
model_name = 'p1-2-bigru-512-true'                    # 模型名称
W_embedding = np.load('../data/W_embedding.npy')            # 导入预训练好的词向量
model_path = '../ckpt/' + model_name + '/'                  # 模型保存位置
summary_path = '../summary/' + model_name + '/'             # summary 位置
result_path = '../result/' + model_name + '.csv'            # result.csv 位置
scores_path = '../scores/' + model_name + '.npy'            # scores.npy 位置

if not os.path.exists(model_path):
    os.makedirs(model_path)         
model_path = model_path + 'model.ckpt'
if os.path.exists(summary_path):   # 删除原来的 summary 文件，避免重合
    shutil.rmtree(summary_path)
os.makedirs(summary_path)          # 然后再次创建


# ##################### config ######################
n_step1 = max_len1 = 50           # 句子长度
n_step2 = max_len2 = 150
input_size = embedding_size = 256       # 字向量长度
n_class = 1999
hidden_size = 512    # 隐含层节点数
fc_hidden_size = 1024                   # fc 层节点数
n_layer = 1        # bi-gru 层数
max_grad_norm = 1.0  # 最大梯度（超过此值的梯度将被裁剪）
seed_num = 13
valid_num = 100000
global_step = 0
tr_batch_size = 128
te_batch_size = 128
print('Prepared, costed time %g s.' % (time.time() - time0))

Starting ...
Prepared, costed time 1.98177 s.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from tensorflow.contrib import rnn
import tensorflow.contrib.layers as layers

'''
双端 GRU，知乎问题多标签分类。
'''

print('Building model ...')
lr = tf.placeholder(tf.float32)
keep_prob = tf.placeholder(tf.float32, [])
batch_size = tf.placeholder(tf.int32, [])  # 注意类型必须为 tf.int32
tst = tf.placeholder(tf.bool)
n_updates = tf.placeholder(tf.int32)      # training iteration,传入 bn 层
update_emas = list()   # BN 层中所有的更新操作


def weight_variable(shape, name):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape, name):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

def batchnorm(Ylogits, is_test, num_updates, offset, convolutional=False):
    """batchnormalization.
    Args:
        Ylogits: 1D向量或者是3D的卷积结果。
        num_updates: 迭代的global_step
        offset：表示beta，全局均值；在 RELU 激活中一般初始化为 0.1。
        scale：表示lambda，全局方差；在 sigmoid 激活中需要，这 RELU 激活中作用不大。
        m: 表示batch均值；v:表示batch方差。
        bnepsilon：一个很小的浮点数，防止除以 0.
    Returns:
        Ybn: 和 Ylogits 的维度一样，就是经过 Batch Normalization 处理的结果。
        update_moving_everages：更新mean和variance，主要是给最后的 test 使用。
    """
    exp_moving_avg = tf.train.ExponentialMovingAverage(0.999, num_updates) # adding the iteration prevents from averaging across non-existing iterations
    bnepsilon = 1e-5
    if convolutional:
        mean, variance = tf.nn.moments(Ylogits, [0, 1, 2])
    else:
        mean, variance = tf.nn.moments(Ylogits, [0])
    update_moving_everages = exp_moving_avg.apply([mean, variance])
    m = tf.cond(is_test, lambda: exp_moving_avg.average(mean), lambda: mean)
    v = tf.cond(is_test, lambda: exp_moving_avg.average(variance), lambda: variance)
    Ybn = tf.nn.batch_normalization(Ylogits, m, v, offset, None, bnepsilon)
    return Ybn, update_moving_everages

# 第一次定义，放在 CPU 上面
with tf.device('/gpu:0'):
    with tf.variable_scope('embedding'):
        embedding = tf.get_variable(name="W_embedding", shape=W_embedding.shape, 
                        initializer=tf.constant_initializer(W_embedding), trainable=True)   # fine-tune
        
with tf.name_scope('Inputs'):
    X1_inputs = tf.placeholder(tf.int64, [None, n_step1], name='X1_input')
    X2_inputs = tf.placeholder(tf.int64, [None, n_step2], name='X2_input')
    y_inputs = tf.placeholder(tf.float32, [None, n_class], name='y_input')
    inputs1 = tf.nn.embedding_lookup(embedding, X1_inputs)
    inputs2 = tf.nn.embedding_lookup(embedding, X2_inputs)

def gru_cell():
    with tf.name_scope('gru_cell'):
        cell = rnn.GRUCell(hidden_size, reuse=tf.get_variable_scope().reuse)
    return rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)


def bi_gru(inputs):
    """build the bi-GRU network. 返回个最后一层的隐含状态。"""      
    cells_fw = [gru_cell() for _ in range(n_layer)]
    cells_bw = [gru_cell() for _ in range(n_layer)]
    initial_states_fw = [cell_fw.zero_state(batch_size, tf.float32) for cell_fw in cells_fw]
    initial_states_bw = [cell_bw.zero_state(batch_size, tf.float32) for cell_bw in cells_bw] 
    outputs, _, _ = rnn.stack_bidirectional_dynamic_rnn(cells_fw, cells_bw, inputs, 
                        initial_states_fw = initial_states_fw, initial_states_bw = initial_states_bw, dtype=tf.float32)
    return outputs
    

def task_specific_attention(inputs, output_size,
                            initializer=layers.xavier_initializer(),
                            activation_fn=tf.tanh, scope=None):
    """
    Performs task-specific attention reduction, using learned
    attention context vector (constant within task of interest).
    Args:
        inputs: Tensor of shape [batch_size, units, input_size]
            `input_size` must be static (known)
            `units` axis will be attended over (reduced from output)
            `batch_size` will be preserved
        output_size: Size of output's inner (feature) dimension
    Returns:
        outputs: Tensor of shape [batch_size, output_dim].
    """
    assert len(inputs.get_shape()) == 3 and inputs.get_shape()[-1].value is not None
    with tf.variable_scope(scope or 'attention') as scope:
        # u_w, attention 向量
        attention_context_vector = tf.get_variable(name='attention_context_vector',
                                                   shape=[output_size],
                                                   initializer=initializer,
                                                   dtype=tf.float32)
        # 全连接层，把 h_i 转为 u_i ， shape= [batch_size, units, input_size] -> [batch_size, units, output_size]
        input_projection = layers.fully_connected(inputs, output_size,
                                                  activation_fn=activation_fn,
                                                  scope=scope)
        # 输出 [batch_size, units]
        vector_attn = tf.reduce_sum(tf.multiply(input_projection, attention_context_vector), axis=2, keep_dims=True)
        attention_weights = tf.nn.softmax(vector_attn, dim=1)   
        tf.summary.histogram('attention_weigths', attention_weights)
        weighted_projection = tf.multiply(inputs, attention_weights)   # ???源代码,原文应该是对 hi 加权！！
        outputs = tf.reduce_sum(weighted_projection, axis=1)
        return outputs   # 输出 [batch_size, hidden_size*2]

    
# 这部分数据需要进行保存
with tf.variable_scope('bigru_title'):          # 两部分的输出都加上 BN+RELU 层，
    word_encoder_title = bi_gru(inputs1)      # title 部分输出
    output_title = task_specific_attention(word_encoder_title, hidden_size*2)
    
with tf.variable_scope('bigru_content'):
    word_encoder_content = bi_gru(inputs2)    # content 部分输出
    output_content = task_specific_attention(word_encoder_content, hidden_size*2)

with tf.name_scope('fc_layer'): # 将两个网络的输出拼接后加一个 fc 层
    output = tf.concat([output_title, output_content], axis=1)
    W_fc = weight_variable([hidden_size*4, fc_hidden_size], name='Weight_fc')
    tf.summary.histogram('W_fc', W_fc)
    h_fc = tf.matmul(output, W_fc, name='h_fc')
    beta_fc = tf.Variable(tf.constant(0.1, tf.float32, shape=[fc_hidden_size], name="beta_fc"))
    tf.summary.histogram('beta_fc', beta_fc)
    fc_bn, update_ema_fc = batchnorm(h_fc, tst, n_updates, beta_fc, convolutional=False)
    update_emas.append(update_ema_fc)
    fc_bn_relu = tf.nn.relu(fc_bn, name="relu")
#     fc_bn_drop = tf.nn.dropout(fc_bn_relu, keep_prob)  # 不用 全连接层
    
    
with tf.name_scope('out_layer'):
    W_out = weight_variable([fc_hidden_size, n_class], name='Weight_out') 
    tf.summary.histogram('Weight_out', W_out)
    b_out = bias_variable([n_class], name='bias_out') 
    tf.summary.histogram('bias_out', b_out)
    y_pred = tf.nn.xw_plus_b(fc_bn_relu, W_out, b_out, name='y_pred')  #每个类别的分数 scores

with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_pred, labels=y_inputs))
    tf.summary.scalar('cost', cost)
    
# ***** 优化求解 *******
with tf.name_scope('AdamOptimizer1'):
    tvars1 = tf.trainable_variables()
    grads1, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars1), max_grad_norm)
    optimizer1 = tf.train.AdamOptimizer(learning_rate=lr)
    train_op1 = optimizer1.apply_gradients(zip(grads1, tvars1),
        global_step=tf.contrib.framework.get_or_create_global_step())
    
with tf.name_scope('AdamOptimizer2'):
    tvars2 = [tvar for tvar in tvars1 if 'embedding' not in tvar.name]
    grads2, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars2), max_grad_norm)
    optimizer2 = tf.train.AdamOptimizer(learning_rate=lr)
    train_op2 = optimizer2.apply_gradients(zip(grads2, tvars2),
        global_step=tf.contrib.framework.get_or_create_global_step())
update_op = tf.group(*update_emas)   # 更新 BN 参数
   
# summary
merged = tf.summary.merge_all() # summary
train_writer = tf.summary.FileWriter(summary_path + 'train', sess.graph)
test_writer = tf.summary.FileWriter(summary_path + 'test')
print 'Finished creating the bi-gru model.'

Building model ...
Finished creating the bi-gru model.


## 导入数据

In [4]:
sys.path.append('..')
from data_helpers import BatchGenerator
from data_helpers import to_categorical
from evaluator import score_eval

save_path = '../data/'
print('loading data...')
time0 = time.time()
X_title = np.load(save_path+'X_tr_title_50.npy')
X_content = np.load(save_path+'X_tr_content_150.npy')
X = np.hstack([X_title, X_content])
y = np.load(save_path+'y_tr.npy')
print('finished loading data, time cost %g s' % (time.time() - time0))
# 划分验证集
sample_num = X.shape[0]
np.random.seed(seed_num)
new_index = np.random.permutation(sample_num)


X = X[new_index]
y = y[new_index]
X_valid = X[:valid_num]
y_valid = y[:valid_num]
X_train = X[valid_num:]
y_train = y[valid_num:]
print('train_num=%d, valid_num=%d' % (X_train.shape[0], X_valid.shape[0]))

# 构建数据生成器
data_train = BatchGenerator(X_train, y_train, shuffle=True)
data_valid = BatchGenerator(X_valid, y_valid, shuffle=False)
print('X_train.shape=', X_train.shape, 'y_train.shape=', y_train.shape)
print('X_valid.shape=', X_valid.shape, 'y_valid.shape=', y_valid.shape)

del X
del y
del X_train
del y_train

# valid 数据及 验证集计算
marked_labels_list = data_valid.y.tolist() # 所有的标注结果
valid_data_size = data_valid.y.shape[0]
def valid_epoch():
    """Testing or valid."""
    data_valid._index_in_epoch = 0  # 先指向第一个值
    _batch_size = te_batch_size
    fetches = [cost, y_pred]   
    batch_num = int(valid_data_size / _batch_size)
    start_time = time.time()
    _costs = 0.0
    predict_labels_list = list()  # 所有的预测结果
    for i in xrange(batch_num):
        X_batch, y_batch = data_valid.next_batch(_batch_size)
        X1_batch = X_batch[:, :n_step1]
        X2_batch = X_batch[:, n_step1:]
        y_batch = to_categorical(y_batch)
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch,  y_inputs:y_batch, lr:1e-4,
                     batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
        _cost, predict_labels = sess.run(fetches, feed_dict)
        _costs += _cost    
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
    precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
    mean_cost = _costs / batch_num
    return mean_cost, precision, recall, f1

print('Finised loading data, time %g s' % (time.time() - time0))

loading data...
finished loading data, time cost 56.9636 s
train_num=2899952, valid_num=100000
('X_train.shape=', (2899952, 200), 'y_train.shape=', (2899952,))
('X_valid.shape=', (100000, 200), 'y_valid.shape=', (100000,))
Finised loading data, time 79.2736 s


## 训练模型

In [4]:
_lr = 8e-4
decay = 0.85
max_epoch = 3           # cpu迭代次数
max_max_epoch = 7       # 最多迭代的次数
valid_step = 10000        # 每 valid_step 就进行一次 valid 运算
tr_batch_num = int(data_train.y.shape[0] / tr_batch_size)  # 每个 epoch 中包含的 batch 数
print('tr_batch_num=%d' % tr_batch_num)
saver = tf.train.Saver(max_to_keep=3)           # 最多保存的模型数量
sess.run(tf.global_variables_initializer())
last_f1 = 0.40
model_num = 0
global_step = 0

tr_batch_num=22655


In [5]:
time0 = time.time()
train_op = train_op2 
for epoch in xrange(max_max_epoch):
    if epoch == max_epoch:
        train_op = train_op1
        if model_num == 0:
            model_num += 1
            save_path = saver.save(sess, model_path, global_step=model_num)
            print('the save path is ', save_path)
        print('Begin updating embedding.')
    print 'EPOCH %d, _lr= %g' % (epoch+1, _lr)
    for batch in xrange(tr_batch_num): 
        global_step += 1
        if (global_step+1) % valid_step == 0:    # 进行 valid 计算
            _lr = _lr*decay
            valid_cost, precision, recall, f1 = valid_epoch()
            print('Global_step=%d: valid cost=%g; p=%g, r=%g, f1=%g, time=%g s' % (
                    global_step, valid_cost, precision, recall, f1, time.time()-time0))
            time0 = time.time()
            if (f1 > last_f1):
                last_f1 = f1
                model_num += 1
                save_path = saver.save(sess, model_path, global_step=model_num)
                print('the save path is ', save_path) 
        X_batch, y_batch = data_train.next_batch(tr_batch_size)
        X1_batch = X_batch[:, :n_step1]
        X2_batch = X_batch[:, n_step1:]
        y_batch = to_categorical(y_batch)
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, y_inputs:y_batch, batch_size:tr_batch_size, lr:_lr,
                     keep_prob:0.5, tst:False, n_updates:global_step}
        fetches = [merged, cost, train_op, update_op]
        summary, _cost, _, _ = sess.run(fetches, feed_dict) # the cost is the mean cost of one batch
        if global_step % 100:
            train_writer.add_summary(summary, global_step)
            X_batch, y_batch = data_valid.next_batch(tr_batch_size)
            X1_batch = X_batch[:, :n_step1]
            X2_batch = X_batch[:, n_step1:]
            y_batch = to_categorical(y_batch)
            feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch,  y_inputs:y_batch,lr:1e-4,
                         batch_size:tr_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
            fetches = [merged, cost]
            summary, _cost = sess.run(fetches, feed_dict)
            test_writer.add_summary(summary, global_step)

valid_cost, precision, recall, f1 = valid_epoch()  # # 每个 epoch 进行一次验证 valid
print('Global_step=%d;  valid cost=%g; p=%g, r=%g, f1=%g; speed=%g s/epoch' % (
    global_step, valid_cost, precision, recall, f1, time.time()-time0) )
if (f1 > last_f1):
    model_num += 1
    save_path = saver.save(sess, model_path, global_step=model_num)
    print('the save path is ', save_path) 

EPOCH 1, _lr= 0.0008
Global_step=9999: valid cost=0.00447596; p=1.20543, r=0.495714, f1=0.351263, time=5363.45 s
Global_step=19999: valid cost=0.00419517; p=1.2934, r=0.53, f1=0.375948, time=5371.25 s
EPOCH 2, _lr= 0.000578
Global_step=29999: valid cost=0.00407777; p=1.33182, r=0.544329, f1=0.386402, time=5379.97 s
Global_step=39999: valid cost=0.00399924; p=1.35518, r=0.553734, f1=0.393108, time=5375.94 s
EPOCH 3, _lr= 0.000417605
Global_step=49999: valid cost=0.00396254; p=1.36594, r=0.557943, f1=0.396134, time=5376.21 s
Global_step=59999: valid cost=0.00393905; p=1.37491, r=0.562066, f1=0.398968, time=5374.34 s
('the save path is ', '../ckpt/p1-2-bigru-512-true/model.ckpt-1')
Begin updating embedding.
EPOCH 4, _lr= 0.00030172
Global_step=69999: valid cost=0.00391393; p=1.38658, r=0.56675, f1=0.40231, time=5459.67 s
('the save path is ', '../ckpt/p1-2-bigru-512-true/model.ckpt-2')
Global_step=79999: valid cost=0.00384138; p=1.41032, r=0.575437, f1=0.408685, time=5749.58 s
('the save 

KeyboardInterrupt: 

In [7]:
valid_cost, precision, recall, f1 = valid_epoch()  # 最后的结果并不好，使用 ckpt-5
print('valid cost=%g; p=%g, r=%g, f1=%g; speed=%g s/epoch' % (valid_cost, precision, recall, f1, time.time()-time0) )

valid cost=0.00384768; p=1.42156, r=0.580501, f1=0.412183; speed=2202.22 s/epoch


## 本地测试

In [5]:
# 导入保存好的模型

from tqdm import tqdm

marked_labels_list = np.load('../data/marked_labels_list.npy')
global_step = 110000

# 导入测试数据
def local_predict(epoch):
    """预测  valid 结果，并保存预测概率 到  scores.csv 文件中。"""
    print('local predicting ...')
    saver = tf.train.Saver()
    best_model_path = model_path + '-' +str(epoch)
    saver.restore(sess, best_model_path)
    time0 = time.time()
    X_te = X_valid
    n_sample = X_te.shape[0]  # 测试样本数量
    _batch_size = 500
    fetches = [y_pred]   
    predict_labels_list = list()  # 所有的预测结果
    predict_scores = list()
    for i in tqdm(xrange(0, n_sample, _batch_size)):
        start = i
        end = start+_batch_size
        if end > n_sample:
            end = n_sample
            _batch_size = end - start
        X_batch = X_te[start:end]
        X1_batch = X_batch[:, :n_step1]
        X2_batch = X_batch[:, n_step1:]
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, lr:1e-5, 
                     batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
        predict_labels = sess.run(fetches, feed_dict)[0]
        predict_scores.append(predict_labels)
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_scores = np.asarray(predict_scores)
    predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
    precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
    print('local valid p=%g, r=%g, f1=%g; speed=%g s/epoch' % ( precision, recall, f1, time.time()-time0) )
#     np.save(local_scores_path, predict_scores)
#     print('Writed the scores into %s, time %g s' % (local_scores_path, time.time()-time0))
    return predict_scores
    
local_scores_path = '../local_scores/' + model_name + '-all.npy'
sum_scores = np.zeros((100000, 1999), dtype=float)   
last_f1 = 0.413398
valid_epoch = [3,4,5]
for epoch in valid_epoch:
    score =  np.vstack(local_predict(epoch))
    sum_scores = sum_scores + score
predict_labels_list = map(lambda label: label.argsort()[-1:-6:-1], sum_scores) # 取最大的5个下标
predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
print('p=%g\tr=%g\tf1=%g;' % ( precision, recall, f1))
if f1 > last_f1:
    np.save(local_scores_path, sum_scores)
    print('save scores to %s' % local_scores_path)

local predicting ...
INFO:tensorflow:Restoring parameters from ../ckpt/p1-2-bigru-512-true/model.ckpt-3


100%|██████████| 200/200 [01:00<00:00,  3.29it/s]


local valid p=1.41032, r=0.575409, f1=0.408671; speed=62.7252 s/epoch
local predicting ...
INFO:tensorflow:Restoring parameters from ../ckpt/p1-2-bigru-512-true/model.ckpt-4


100%|██████████| 200/200 [01:00<00:00,  3.35it/s]


local valid p=1.42157, r=0.580262, f1=0.412064; speed=64.5016 s/epoch
local predicting ...
INFO:tensorflow:Restoring parameters from ../ckpt/p1-2-bigru-512-true/model.ckpt-5


100%|██████████| 200/200 [00:59<00:00,  3.33it/s]


local valid p=1.4263, r=0.582079, f1=0.413378; speed=62.7891 s/epoch
p=1.43282	r=0.584576	f1=0.415185;
save scores to ../local_scores/p1-2-bigru-512-true-all.npy


## 对测试数据进行预测

In [6]:
# 导入保存好的模型
# saver = tf.train.Saver()
# best_model_path = model_path + '-' + str(6)  # 导入最优模型
# saver.restore(sess, best_model_path)
# print('Finished loading model.')


# 导入测试数据
def predict(epoch):
    """预测测试集结果，并保存到  result.csv 文件中。"""
    print('local predicting ...')
    saver = tf.train.Saver()
    best_model_path = model_path + '-' + str(epoch)  # 导入最优模型
    saver.restore(sess, best_model_path)
    print('Finished loading model.')
    X1_te = np.load('../data/X_te_title_50.npy')
    X2_te = np.load('../data/X_te_content_150.npy')
    X_te = np.hstack([X1_te, X2_te])
    n_sample = X_te.shape[0]  # 测试样本数量
    _batch_size = 500
    fetches = [y_pred]   
    predict_labels_list = list()  # 所有的预测结果
    predict_scores = list()
    for i in tqdm(xrange(0, n_sample, _batch_size)):
        start = i
        end = start+_batch_size
        if end > n_sample:
            end = n_sample
            _batch_size = end - start
        X_batch = X_te[start:end]
        X1_batch = X_batch[:, :n_step1]
        X2_batch = X_batch[:, n_step1:]
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, lr:1e-5, batch_size:_batch_size, 
                     keep_prob:1.0, tst:True, n_updates:global_step}
        predict_labels = sess.run(fetches, feed_dict)[0]
        predict_scores.append(predict_labels)
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_scores = np.asarray(predict_scores)
    return np.vstack(predict_scores)

sum_scores = np.zeros((217360, 1999), dtype=float)   
valid_epoch = [3,4,5]
for epoch in valid_epoch:
    score = predict(epoch)
    sum_scores = sum_scores + score
print('sum_scores.shape=', sum_scores.shape)
np.save('../scores/' + model_name + '-all.npy', sum_scores)
print('Finished.')

local predicting ...
INFO:tensorflow:Restoring parameters from ../ckpt/p1-2-bigru-512-true/model.ckpt-3
Finished loading model.


100%|██████████| 435/435 [02:11<00:00,  3.67it/s]


local predicting ...
INFO:tensorflow:Restoring parameters from ../ckpt/p1-2-bigru-512-true/model.ckpt-4
Finished loading model.


100%|██████████| 435/435 [02:11<00:00,  3.45it/s]


local predicting ...
INFO:tensorflow:Restoring parameters from ../ckpt/p1-2-bigru-512-true/model.ckpt-5
Finished loading model.


100%|██████████| 435/435 [02:12<00:00,  3.47it/s]


('sum_scores.shape=', (217360, 1999))
Finished.


In [9]:
sc = np.load('../local_scores/p1-1-bigru-512-all.npy')
print(sc.shape)

(100000, 1999)


In [12]:
# 预测
time0 = time.time()
print('Predicting the result, writing into %s' % result_path)
predict_labels_list,predict_scores = predict()
df_result = write_result(predict_labels_list, result_path=result_path) 
print(len(df_result))  # 结果应该为 217360
print('Saving the predict_scores into %s' % scores_path)
print(predict_scores.shape)
np.save(scores_path, predict_scores)
print('Finished! costed time %g s' % (time.time()-time0))

Predicting the result, writing into ../result/p1-2-bigru-512-true.csv
Predicting ...
Finished writing the result
217360
Saving the predict_scores into ../scores/p1-2-bigru-512-true.npy
(217360, 1999)
Finished! costed time 155.846 s


In [13]:
print(np.vstack(predict_scores).shape)
print(predict_scores[0].shape)
print(predict_scores[0])

(217360, 1999)
(1999,)
[ -6.66303682  -7.70492887  -8.22773075 ..., -12.6364336  -24.37023354
 -11.60172367]


In [14]:
df_result.head(5)

,question,tid0,tid1,tid2,tid3,tid4
0,6215603645409872328,-7506384235581390893,4610596224687453206,2919247920214845195,-6839713564940654454,-6306904715218704629
1,6649324930261961840,-240041917918953337,2858911571784840089,3418451812342379591,3383016985780045156,2382911985227044227
2,-4251899610700378615,2919247920214845195,-7358589937244777363,2816249700493135244,-3315241959305847628,-5265476641576484497
3,6213817087034420233,-8655945395761165989,5804619920623030604,-4966205278807386328,244937959911721367,7476760589625268543
4,-8930652370334418373,3972493657017129406,-8963554618409314978,-1115593437686158905,7951349602759061249,6018641953300645757


In [11]:
# 参考结果
df_result.head(5)

,question,tid0,tid1,tid2,tid3,tid4
0,6215603645409872328,4610596224687453206,-6839713564940654454,5818382893362053755,-7506384235581390893,-5932391056759866388
1,6649324930261961840,3418451812342379591,2858911571784840089,-240041917918953337,2382911985227044227,-7483543763655495143
2,-4251899610700378615,2919247920214845195,-7358589937244777363,2816249700493135244,-7270992690764838239,-3315241959305847628
3,6213817087034420233,5804619920623030604,-4966205278807386328,-8655945395761165989,244937959911721367,7476760589625268543
4,-8930652370334418373,3972493657017129406,-8963554618409314978,-1115593437686158905,-6925670792665757873,6018641953300645757


## 在全部预测正确的情况下，理论值为：f1=0.713933
precision=2.50273, recall=0.998873, f1=0.713933

In [26]:
# 假设全部正确，f1 值最高能到多少
def padding_label(labels):
    """把所有的label补齐到长度为 5"""
    label_len = len(labels)
    if label_len >= 5:
        return labels[:5]
    return np.hstack([labels, np.zeros(5-label_len, dtype=int) - 1])
    

marked_labels_list = data_valid.y.tolist() # 所有的标注结果
predict_labels_list = map(padding_label, marked_labels_list)
predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
print '在全部预测正确的情况下，理论值为：'
print 'precision=%g, recall=%g, f1=%g' % (precision, recall, f1)

在全部预测正确的情况下，理论值为：
precision=2.50273, recall=0.998873, f1=0.713933
